In [0]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

import json
from glob import glob
from pathlib import Path
import os


In [66]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [67]:
%env TRAC_PATH /content/gdrive/My Drive/TRAC2020-master/
TRAC_PATH = "/content/gdrive/My Drive/TRAC2020-master"
os.chdir(TRAC_PATH)
os.listdir()

env: TRAC_PATH=/content/gdrive/My Drive/TRAC2020-master/


['HIN',
 'IBEN',
 'notebooks',
 'src',
 'ALL',
 'data',
 'ENG',
 '.gitignore',
 'LICENSE',
 'system_paper_figures_and_tables.py',
 'create_run_submissions.py',
 'README.md',
 'results.csv']

In [0]:
lang = ["./ENG","./HIN","./IBEN","./ALL"]
filepaths=[]
#print(os.getcwd())
#str(os.path.join(lang[0],"**/output/**/*_results.json"))
for l in lang:
    for v in glob(str(os.path.join(".",l,"**/output/**/*_results.json"))):
        #print(str(os.path.join("./",l,"/**/output/**/*_results.json")))
        filepaths.append(Path(v))
#filepaths = [Path(v) for v in glob(os.path.join("./",lang,"/**/output/**/*_results.json"))]

In [69]:
filepaths

[PosixPath('ENG/Sub-task B/output/xlm-roberta-base/train_results.json'),
 PosixPath('ENG/Sub-task B/output/xlm-roberta-base/dev_results.json'),
 PosixPath('ENG/Sub-task B/output/bert-base-cased/dev_results.json'),
 PosixPath('ENG/Sub-task B/output/bert-base-cased/train_results.json'),
 PosixPath('ENG/Sub-task B/output/bert-base-uncased/train_results.json'),
 PosixPath('ENG/Sub-task B/output/bert-base-uncased/dev_results.json'),
 PosixPath('ENG/Sub-task C/output/bert-base-uncased/train_results.json'),
 PosixPath('ENG/Sub-task C/output/bert-base-uncased/dev_results.json'),
 PosixPath('ENG/Sub-task C/output/bert-base-cased/train_results.json'),
 PosixPath('ENG/Sub-task C/output/bert-base-cased/dev_results.json'),
 PosixPath('ENG/Sub-task A/output/bert-base-uncased/dev_results.json'),
 PosixPath('ENG/Sub-task A/output/bert-base-uncased/train_results.json'),
 PosixPath('ENG/Sub-task A/output/xlm-roberta-base/train_results.json'),
 PosixPath('ENG/Sub-task A/output/xlm-roberta-base/dev_result

In [70]:
def get_filestats(filepath):
    stats = []
    with open(filepath) as fp:
        for line in fp:
            stats.append(json.loads(line))
    return stats

stats = get_filestats(filepaths[0])
len(stats)


66

In [0]:
TASK_MAPPING={
    "Sub-task A": "A",
    "Sub-task B": "B",
    "Sub-task C": "C",
}

MODEL_DETAILS = {
    ("ENG", "A", "bert-base-cased"): ("1"),
    ("ENG", "B", "bert-base-cased"): ("1"),
    ("ENG", "C", "bert-base-cased"): ("3"),

    ("ENG", "A", "bert-base-uncased"): ("2"),
    ("ENG", "B", "bert-base-uncased"): ("2"),
    ("ENG", "C", "bert-base-uncased"): ("4"),

    ("ENG", "A", "xlm-roberta-base"): ("5"),
    ("ENG", "B", "xlm-roberta-base"): ("5"),
    ("ENG", "C", "xlm-roberta-base"): ("6"),

    ("ENG", "A", "bert-base-multilingual-uncased"): ("7"),
    ("ENG", "B", "bert-base-multilingual-uncased"): ("8"),

    ("HIN", "A", "bert-base-multilingual-uncased"): ("1"),
    ("HIN", "B", "bert-base-multilingual-uncased"): ("1"),
    ("HIN", "C", "bert-base-multilingual-uncased"): ("3"),

    ("HIN", "A", "xlm-roberta-base"): ("2"),
    ("HIN", "B", "xlm-roberta-base"): ("2"),
    ("HIN", "C", "xlm-roberta-base"): ("4"),

    ("HIN", "A", "bert-base-multilingual-uncased"): ("5"),
    ("HIN", "B", "bert-base-multilingual-uncased"): ("6"),

    ("IBEN", "A", "bert-base-multilingual-uncased"): ("1"),
    ("IBEN", "B", "bert-base-multilingual-uncased"): ("1"),
    ("IBEN", "C", "bert-base-multilingual-uncased"): ("3"),

    ("IBEN", "A", "xlm-roberta-base"): ("2"),
    ("IBEN", "B", "xlm-roberta-base"): ("2"),
    ("IBEN", "C", "xlm-roberta-base"): ("4"),

    ("IBEN", "A", "bert-base-multilingual-uncased"): ("5"),
    ("IBEN", "B", "bert-base-multilingual-uncased"): ("6"),
    
}


In [0]:
def get_model_C_stats(filepath, lang, model, task, split, run_id):
    print(filepath, lang, model, task, split, run_id)
    base_path = filepath.parent
    final_path = base_path / filepath.name.replace("_results.json", ".tsv")
    model = f"{model}"
    #run_id = run_id.split(" ") if run_id else []
    #base_run_id = ""
    #if len(run_id) == 2:
    #    base_run_id = run_id[0]
    #    run_id_tasks = run_id[1][1:-1]
    #    run_id_tasks = {1,2,3} if run_id_tasks == "all" else {int(run_id_tasks),}
    #elif len(run_id) == 0:
    #    run_id_tasks = {}
    #else:
    #    raise RuntimeError(f"Unknown run_id={run_id}")

    df = pd.read_csv(final_path, sep="\t")
    lblsA = ['OAG','NAG','CAG']
    lblsB = ['GEN','NGEN']
    Albls = {i:l for i,l in enumerate(lblsA)}
    Blbls = {i:l for i,l in enumerate(lblsB)}
    for i in lblsA:
        df[i+'_probs'] = df[i+'-'+lblsB[0]+'_probs']+df[i+'-'+lblsB[1]+'_probs']  
    for i in lblsB:
        df[i+'_probs'] = df[lblsA[0]+'-'+i+'_probs']+df[lblsA[1]+'-'+i+'_probs']+df[lblsA[2]+'-'+i+'_probs']  
    a = pd.DataFrame({'val':np.argmax( [df[lblsA[0]+'_probs'] , df[lblsA[1]+'_probs'], df[lblsA[2]+'_probs']],axis=0)})
    b = pd.DataFrame({'val':np.argmax([df[lblsB[0]+'_probs'], df[lblsB[1]+'_probs']],axis=0)})
    df["A_preds"] = a['val'].map(Albls)
    df["B_preds"] = b['val'].map(Blbls)
    df["Sub-task C_mpreds"] = df["A_preds"]+'-'+df["B_preds"]

    tmp = {
    0 : "Sub-task C_preds",
    1 : "Sub-task C_mpreds"
    } 

    tmp1 = {
    0 : " (C)",
    1 : " (M)"
    }
    
    for j in range(2):
        for i, task in enumerate(["A", "B"]):
            #run_id = ""
        #    if lang == "DE" and task == "C":
        #        continue
        #    if i+1 in run_id_tasks:
        #        run_id = base_run_id
            report = classification_report(
                df["label"].str.split("-", expand=True)[i],
                df[tmp[j]].str.split("-", expand=True)[i],
                output_dict=True
            )
            final_stat = {
                "macro avg/f1-score": report["macro avg"]["f1-score"],
                "weighted avg/f1-score": report["weighted avg"]["f1-score"]
            }
            final_stat.update(dict(lang=lang, model=model+tmp1[j], task=task, split=split, run_id=run_id+tmp1[j]))
            yield final_stat

def get_model_ALL_stats(filepath, lang, model, task, split, run_id):
    print(filepath, lang, model, task, split, run_id)
    base_path = filepath.parent
    final_path = base_path / filepath.name.replace("_results.json", ".tsv")
    model = f"{model}"
    #run_id = run_id.split(" ") if run_id else []
    #base_run_id = ""
    #if len(run_id) == 2:
    #    base_run_id = run_id[0]
    #    run_id_tasks = run_id[1][1:-1]
    #    run_id_tasks = {1,2,3} if run_id_tasks == "all" else {int(run_id_tasks),}
    #elif len(run_id) == 0:
    #    run_id_tasks = {}
    #else:
    #    raise RuntimeError(f"Unknown run_id={run_id}")

    df = pd.read_csv(final_path, sep="\t")
    if task != "Sub-task C":        
        for i, l in enumerate(["ENG", "HIN", "IBEN"]):
            #run_id = ""
        #    if lang == "DE" and task == "C":
        #        continue
        #    if i+1 in run_id_tasks:
        #        run_id = base_run_id
            pred_label = "Sub-task "+task+"_preds"
            actual_label = "label"
            report = classification_report(
                df.loc[df["id"].str.split('-',expand=True)[0]==l,actual_label],
                #df["label"].str.split("-", expand=True)[i],
                df.loc[df["id"].str.split('-',expand=True)[0]==l,pred_label],
                #df["Sub-task C_preds"].str.split("-", expand=True)[i],
                output_dict=True
            )
            final_stat = {
                "macro avg/f1-score": report["macro avg"]["f1-score"],
                "weighted avg/f1-score": report["weighted avg"]["f1-score"]
            }
            final_stat.update(dict(lang=l, model=model, task=task, split=split, run_id=run_id))
            yield final_stat
        

def get_final_stats(filepath):
    lang, task, _, model, basename = filepath.parts[:]
    stats = get_filestats(filepath)
    split = basename.split("_")[0]
    task = TASK_MAPPING[task]
    default_model_details = ("", )
    run_id, = MODEL_DETAILS.get((lang, task, model), default_model_details)
    if lang == "ALL":
        run_id = 9
        return [
            final_stat
            for final_stat in get_model_ALL_stats(
                filepath, lang=lang, model=model+" (ALL)", task=task, split=split, run_id=run_id
            )
        ]
    if task == "C":
        #print("C\n")
        return [
            final_stat
            for final_stat in get_model_C_stats(
                filepath, lang=lang, model=model+" (C)", task=task, split=split, run_id=run_id
            )
        ]
    final_stat = stats[-1]
    final_stat.update(dict(lang=lang, model=model, task=task, split=split, run_id=run_id))
    #print(lang,model,task)
    return [final_stat]


In [73]:
all_stats = sum((get_final_stats(filepath) for filepath in filepaths), [])
len(all_stats)
all_stats

ENG/Sub-task C/output/bert-base-uncased/train_results.json ENG bert-base-uncased (C) C train 4
ENG/Sub-task C/output/bert-base-uncased/dev_results.json ENG bert-base-uncased (C) C dev 4
ENG/Sub-task C/output/bert-base-cased/train_results.json ENG bert-base-cased (C) C train 3
ENG/Sub-task C/output/bert-base-cased/dev_results.json ENG bert-base-cased (C) C dev 3
HIN/Sub-task C/output/bert-base-multilingual-uncased/train_results.json HIN bert-base-multilingual-uncased (C) C train 3
HIN/Sub-task C/output/bert-base-multilingual-uncased/dev_results.json HIN bert-base-multilingual-uncased (C) C dev 3
IBEN/Sub-task C/output/bert-base-multilingual-uncased/train_results.json IBEN bert-base-multilingual-uncased (C) C train 3
IBEN/Sub-task C/output/bert-base-multilingual-uncased/dev_results.json IBEN bert-base-multilingual-uncased (C) C dev 3
ALL/Sub-task A/output/bert-base-multilingual-uncased/train_results.json ALL bert-base-multilingual-uncased (ALL) A train 9
ALL/Sub-task A/output/bert-base-m

[{'GEN/f1-score': 0.8421052631578948,
  'GEN/precision': 0.8857142857142857,
  'GEN/recall': 0.8025889967637541,
  'GEN/support': 309,
  'NGEN/f1-score': 0.9882827264709588,
  'NGEN/precision': 0.9846849108712026,
  'NGEN/recall': 0.9919069296914517,
  'NGEN/support': 3954,
  'accuracy': 0.9781843771991555,
  'eval_loss': 0.08544123072813196,
  'global_step': 1320,
  'lang': 'ENG',
  'loss': 0.07974085295572877,
  'macro avg/f1-score': 0.9151939948144268,
  'macro avg/precision': 0.9351995982927441,
  'macro avg/recall': 0.8972479632276029,
  'macro avg/support': 4263,
  'model': 'xlm-roberta-base',
  'run_id': '5',
  'split': 'train',
  'task': 'B',
  'weighted avg/f1-score': 0.9776871749429886,
  'weighted avg/precision': 0.9775111076402648,
  'weighted avg/recall': 0.9781843771991555,
  'weighted avg/support': 4263},
 {'GEN/f1-score': 0.5,
  'GEN/precision': 0.5818181818181818,
  'GEN/recall': 0.4383561643835616,
  'GEN/support': 73,
  'NGEN/f1-score': 0.968063872255489,
  'NGEN/pre

In [74]:
df_map = {
    k: pd.DataFrame([stats for stats in all_stats if stats["task"] == k])
    for k in ["A", "B",]
}
df_map["A"].head()

,macro avg/f1-score,weighted avg/f1-score,lang,model,task,split,run_id,eval_loss,OAG/precision,OAG/recall,OAG/f1-score,OAG/support,NAG/precision,NAG/recall,NAG/f1-score,NAG/support,CAG/precision,CAG/recall,CAG/f1-score,CAG/support,accuracy,macro avg/precision,macro avg/recall,macro avg/support,weighted avg/precision,weighted avg/recall,weighted avg/support,global_step,loss
0,0.901949,0.956099,ENG,bert-base-uncased (C) (C),A,train,4 (C),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.900385,0.955688,ENG,bert-base-uncased (C) (M),A,train,4 (M),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.595569,0.794969,ENG,bert-base-uncased (C) (C),A,dev,4 (C),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.595146,0.794839,ENG,bert-base-uncased (C) (M),A,dev,4 (M),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.911844,0.960905,ENG,bert-base-cased (C) (C),A,train,3 (C),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [75]:
with pd.option_context("precision", 3):
    for k in ["A", "B"]:
        print(f"subtask={k}")
        df_t = df_map[k][["lang", "task", "model", "run_id", "split", "macro avg/f1-score", "weighted avg/f1-score"]].rename(columns={
            "macro avg/f1-score": "macro", "weighted avg/f1-score": "weighted"
        }).pivot_table(index=["lang", "task", "model", "run_id"], columns=["split"])
        display(df_t)

subtask=A


macro  ... weighted
split                                                      dev  ...    train
lang task model                                  run_id         ...         
ENG  A    bert-base-cased                        1       0.563  ...    0.987
          bert-base-cased (C) (C)                3 (C)   0.571  ...    0.961
          bert-base-cased (C) (M)                3 (M)   0.568  ...    0.960
          bert-base-multilingual-uncased (ALL)   9       0.611  ...    0.957
          bert-base-uncased                      2       0.577  ...    0.979
          bert-base-uncased (C) (C)              4 (C)   0.596  ...    0.956
          bert-base-uncased (C) (M)              4 (M)   0.595  ...    0.956
          xlm-roberta-base                       5       0.531  ...    0.862
HIN  A    bert-base-multilingual-uncased         5       0.637  ...    0.881
          bert-base-multilingual-uncased (ALL)   9       0.626  ...    0.952
          bert-base-multilingual-uncased (C) (C) 3 (C)   0.616  ...    0.884
          bert-base-multilingual-uncased (C) (M) 3 (M)   0.611  ...    0.884
          xlm-roberta-base                       2       0.394  ...    0.509
IBEN A    bert-base-multilingual-uncased         5       0.680  ...    0.918
          bert-base-multilingual-uncased (ALL)   9       0.698  ...    0.945
          bert-base-multilingual-uncased (C) (C) 3 (C)   0.684  ...    0.883
          bert-base-multilingual-uncased (C) (M) 3 (M)   0.686  ...    0.879
          xlm-roberta-base                       2       0.584  ...    0.691

[18 rows x 4 columns]

subtask=B


macro  ... weighted
split                                                      dev  ...    train
lang task model                                  run_id         ...         
ENG  B    bert-base-cased                        1       0.700  ...    0.995
          bert-base-cased (C) (C)                3 (C)   0.729  ...    0.982
          bert-base-cased (C) (M)                3 (M)   0.727  ...    0.983
          bert-base-multilingual-uncased (ALL)   9       0.751  ...    0.996
          bert-base-uncased                      2       0.737  ...    0.998
          bert-base-uncased (C) (C)              4 (C)   0.734  ...    0.977
          bert-base-uncased (C) (M)              4 (M)   0.757  ...    0.978
          xlm-roberta-base                       5       0.734  ...    0.978
HIN  B    bert-base-multilingual-uncased         6       0.780  ...    0.986
          bert-base-multilingual-uncased (ALL)   9       0.778  ...    0.994
          bert-base-multilingual-uncased (C) (C) 3 (C)   0.750  ...    0.917
          bert-base-multilingual-uncased (C) (M) 3 (M)   0.760  ...    0.916
          xlm-roberta-base                       2       0.459  ...    0.759
IBEN B    bert-base-multilingual-uncased         6       0.830  ...    0.985
          bert-base-multilingual-uncased (ALL)   9       0.849  ...    0.992
          bert-base-multilingual-uncased (C) (C) 3 (C)   0.824  ...    0.953
          bert-base-multilingual-uncased (C) (M) 3 (M)   0.827  ...    0.954
          xlm-roberta-base                       2       0.714  ...    0.855

[18 rows x 4 columns]

In [76]:
with pd.option_context("precision", 3):
    df_t = pd.concat([df_map[k][["lang", "task", "model", "run_id", "split", "macro avg/f1-score", "weighted avg/f1-score"]].rename(columns={
            "macro avg/f1-score": "macro", "weighted avg/f1-score": "weighted"
        }) for k in ["A", "B"]])
    df_t = df_t.pivot_table(index=["task", "lang", "model", "run_id"], columns=["split"])
    display(df_t)

macro  ... weighted
split                                                      dev  ...    train
task lang model                                  run_id         ...         
A    ENG  bert-base-cased                        1       0.563  ...    0.987
          bert-base-cased (C) (C)                3 (C)   0.571  ...    0.961
          bert-base-cased (C) (M)                3 (M)   0.568  ...    0.960
          bert-base-multilingual-uncased (ALL)   9       0.611  ...    0.957
          bert-base-uncased                      2       0.577  ...    0.979
          bert-base-uncased (C) (C)              4 (C)   0.596  ...    0.956
          bert-base-uncased (C) (M)              4 (M)   0.595  ...    0.956
          xlm-roberta-base                       5       0.531  ...    0.862
     HIN  bert-base-multilingual-uncased         5       0.637  ...    0.881
          bert-base-multilingual-uncased (ALL)   9       0.626  ...    0.952
          bert-base-multilingual-uncased (C) (C) 3 (C)   0.616  ...    0.884
          bert-base-multilingual-uncased (C) (M) 3 (M)   0.611  ...    0.884
          xlm-roberta-base                       2       0.394  ...    0.509
     IBEN bert-base-multilingual-uncased         5       0.680  ...    0.918
          bert-base-multilingual-uncased (ALL)   9       0.698  ...    0.945
          bert-base-multilingual-uncased (C) (C) 3 (C)   0.684  ...    0.883
          bert-base-multilingual-uncased (C) (M) 3 (M)   0.686  ...    0.879
          xlm-roberta-base                       2       0.584  ...    0.691
B    ENG  bert-base-cased                        1       0.700  ...    0.995
          bert-base-cased (C) (C)                3 (C)   0.729  ...    0.982
          bert-base-cased (C) (M)                3 (M)   0.727  ...    0.983
          bert-base-multilingual-uncased (ALL)   9       0.751  ...    0.996
          bert-base-uncased                      2       0.737  ...    0.998
          bert-base-uncased (C) (C)              4 (C)   0.734  ...    0.977
          bert-base-uncased (C) (M)              4 (M)   0.757  ...    0.978
          xlm-roberta-base                       5       0.734  ...    0.978
     HIN  bert-base-multilingual-uncased         6       0.780  ...    0.986
          bert-base-multilingual-uncased (ALL)   9       0.778  ...    0.994
          bert-base-multilingual-uncased (C) (C) 3 (C)   0.750  ...    0.917
          bert-base-multilingual-uncased (C) (M) 3 (M)   0.760  ...    0.916
          xlm-roberta-base                       2       0.459  ...    0.759
     IBEN bert-base-multilingual-uncased         6       0.830  ...    0.985
          bert-base-multilingual-uncased (ALL)   9       0.849  ...    0.992
          bert-base-multilingual-uncased (C) (C) 3 (C)   0.824  ...    0.953
          bert-base-multilingual-uncased (C) (M) 3 (M)   0.827  ...    0.954
          xlm-roberta-base                       2       0.714  ...    0.855

[36 rows x 4 columns]

In [77]:
with pd.option_context("precision", 3):
    df_t = pd.concat([df_map[k][["lang", "task", "model", "run_id", "split", "macro avg/f1-score", "weighted avg/f1-score"]].rename(columns={
            "macro avg/f1-score": "macro", "weighted avg/f1-score": "weighted"
        }) for k in ["A", "B"]])
    df_t = df_t.pivot_table(index=["lang", "task", "model", "run_id"], columns=["split"])
    display(df_t)

macro  ... weighted
split                                                      dev  ...    train
lang task model                                  run_id         ...         
ENG  A    bert-base-cased                        1       0.563  ...    0.987
          bert-base-cased (C) (C)                3 (C)   0.571  ...    0.961
          bert-base-cased (C) (M)                3 (M)   0.568  ...    0.960
          bert-base-multilingual-uncased (ALL)   9       0.611  ...    0.957
          bert-base-uncased                      2       0.577  ...    0.979
          bert-base-uncased (C) (C)              4 (C)   0.596  ...    0.956
          bert-base-uncased (C) (M)              4 (M)   0.595  ...    0.956
          xlm-roberta-base                       5       0.531  ...    0.862
     B    bert-base-cased                        1       0.700  ...    0.995
          bert-base-cased (C) (C)                3 (C)   0.729  ...    0.982
          bert-base-cased (C) (M)                3 (M)   0.727  ...    0.983
          bert-base-multilingual-uncased (ALL)   9       0.751  ...    0.996
          bert-base-uncased                      2       0.737  ...    0.998
          bert-base-uncased (C) (C)              4 (C)   0.734  ...    0.977
          bert-base-uncased (C) (M)              4 (M)   0.757  ...    0.978
          xlm-roberta-base                       5       0.734  ...    0.978
HIN  A    bert-base-multilingual-uncased         5       0.637  ...    0.881
          bert-base-multilingual-uncased (ALL)   9       0.626  ...    0.952
          bert-base-multilingual-uncased (C) (C) 3 (C)   0.616  ...    0.884
          bert-base-multilingual-uncased (C) (M) 3 (M)   0.611  ...    0.884
          xlm-roberta-base                       2       0.394  ...    0.509
     B    bert-base-multilingual-uncased         6       0.780  ...    0.986
          bert-base-multilingual-uncased (ALL)   9       0.778  ...    0.994
          bert-base-multilingual-uncased (C) (C) 3 (C)   0.750  ...    0.917
          bert-base-multilingual-uncased (C) (M) 3 (M)   0.760  ...    0.916
          xlm-roberta-base                       2       0.459  ...    0.759
IBEN A    bert-base-multilingual-uncased         5       0.680  ...    0.918
          bert-base-multilingual-uncased (ALL)   9       0.698  ...    0.945
          bert-base-multilingual-uncased (C) (C) 3 (C)   0.684  ...    0.883
          bert-base-multilingual-uncased (C) (M) 3 (M)   0.686  ...    0.879
          xlm-roberta-base                       2       0.584  ...    0.691
     B    bert-base-multilingual-uncased         6       0.830  ...    0.985
          bert-base-multilingual-uncased (ALL)   9       0.849  ...    0.992
          bert-base-multilingual-uncased (C) (C) 3 (C)   0.824  ...    0.953
          bert-base-multilingual-uncased (C) (M) 3 (M)   0.827  ...    0.954
          xlm-roberta-base                       2       0.714  ...    0.855

[36 rows x 4 columns]

In [0]:
from google.colab import files
df_t.to_csv('results.csv') 
files.download('results.csv')